In [16]:
import json
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

In [9]:
image_path = 'datasets/train/image'
annotation_path = 'datasets/train/annos'

In [32]:
color_map = {
    0: [0, 0, 0],        # Background: Black
    1: [0, 128, 0],      # Trousers: Dark Green
    2: [255, 0, 0],      # Short Sleeve Top: Red
    3: [0, 0, 255],      # Long Sleeve Top: Blue
    4: [0, 255, 0],      # Skirt: Lime
    5: [255, 255, 0],    # Dress: Yellow
    6: [255, 165, 0],    # Jacket: Orange
    7: [255, 192, 203],  # Blouse: Pink
    8: [128, 0, 128],    # Suit: Purple
    9: [0, 255, 255],    # Coat: Cyan
    10: [128, 128, 0],   # Vest: Olive
    11: [255, 0, 255],   # Shirt: Magenta
    12: [0, 128, 128],   # Sweater: Teal
    13: [128, 128, 128], # Hoodie: Gray
    14: [0, 255, 255],   # Jumpsuit: Aqua
    15: [255, 255, 255], # Cardigan: White
    16: [128, 0, 0],     # Pants: Maroon
    17: [255, 105, 180], # Shorts: Hot Pink
    18: [105, 105, 105], # Sweatpants: Dim Gray
    19: [173, 216, 230]  # Overalls: Light Blue
}

In [26]:
def apply_color_map(mask, color_map):
    # Convert mask to RGB image using the color map
    rgb_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
    for label, color in color_map.items():
        rgb_mask[mask == label] = color
    return rgb_mask

In [27]:
def overlay_mask_on_image(image, mask, color_map):
    # Convert mask to RGB format
    mask_rgb = apply_color_map(mask, color_map)
    
    # Overlay mask on the image
    overlay = cv2.addWeighted(image, 0.7, mask_rgb, 0.3, 0)
    return overlay

In [12]:
def load_json(json_path):
    with open(json_path, 'r') as file:
        data = json.load(file)
    return data

In [19]:
exclude_keys = ['source','pair_id']
def create_mask_for_whole_image(image_shape, segmentation_cats):
    mask = np.zeros((image_shape[0], image_shape[1]), dtype=np.int8)
    for data in segmentation_cats:
        segmentation = data['data_points']
        class_label = data['class_label']
        for seg in segmentation:
            points = np.array(seg).reshape(-1, 2)
            cv2.fillPoly(mask, [points.astype(np.int32)], class_label)
    return mask

In [23]:
def build_mask(image_shape,json_data):
    image_segmentation_data_points = []
    filtered_keys = [key for key in json_data.keys() if key not in exclude_keys]
    for key in filtered_keys:
        segmentation_with_class = {'data_points': json_data[key]['segmentation'],
                                   'category':json_data[key]['category_name'],
                                   'class_label':json_data[key]['category_id']}
        image_segmentation_data_points.append(segmentation_with_class)
    mask = create_mask_for_whole_image(image_shape,image_segmentation_data_points)
    return mask

In [49]:
image_count_to_save = 100
image_annot_save_path = 'new'
output_image_shape = (256,256)
for file in sorted(os.listdir(image_path))[0:100]:
    full_path = os.path.join(image_path, file)
    annot_path = os.path.join(annotation_path, file.split('.')[0]+'.json')
    annot_data = load_json(annot_path)
    image = cv2.imread(full_path)
    # image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mask = build_mask(image.shape,annot_data)
    resized_image = cv2.resize(image, output_image_shape, interpolation = cv2.INTER_NEAREST)
    resized_mask = cv2.resize(mask, output_image_shape, interpolation = cv2.INTER_NEAREST)
    image_save_path = os.path.join(image_annot_save_path, file)
    annot_save_path = os.path.join(image_annot_save_path, 'annot-'+file)
    cv2.imwrite(image_save_path, resized_image)
    cv2.imwrite(annot_save_path, resized_mask)
    # overlay = overlay_mask_on_image(image_rgb, mask, color_map)
    # plt.imshow(overlay)